In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [4]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
print(df.shape, test.shape)
df.head()

In [ ]:
#Разделение данных
X = df.iloc[:,:-1]
y = df['label']
print(X.shape, y.shape)
X.head()

In [15]:
#Заполнение пропущенных значений
"""
X.fillna(inplace=True, method='bfill')
X.fillna(inplace=True, method='ffill')
test.fillna(inplace=True, method='bfill')
test.fillna(inplace=True, method='ffill')
X = X.iloc[:30000, :-1]
print(X.shape)
X.head()
"""

"\nX.fillna(inplace=True, method='bfill')\nX.fillna(inplace=True, method='ffill')\ntest.fillna(inplace=True, method='bfill')\ntest.fillna(inplace=True, method='ffill')\nX = X.iloc[:30000, :-1]\nprint(X.shape)\nX.head()\n"

In [7]:
#Посмотрим сколько всего пропущенных значений
print(X.dropna().shape, test.dropna().shape)
#из 50000 строк, только 1765 не имеют пропусков на тренировочных даных
# и 385 из 10000 на тестовых данных
# значит пропуски нужно "хорошо" заполнить

(1765, 111) (385, 111)


In [16]:
#Заполнение пропусков средним значением по столбцу
"""
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X = pd.DataFrame(imp.fit_transform(X))
test = pd.DataFrame(imp.fit_transform(test))
"""

In [8]:

from sklearn.impute import KNNImputer
imp = KNNImputer(n_neighbors=3)
X = pd.DataFrame(imp.fit_transform(X), columns=X.columns)


In [10]:
imp = KNNImputer(n_neighbors=4)
test = pd.read_csv('test.csv')
test = pd.DataFrame(imp.fit_transform(test), columns=test.columns)

In [11]:
X.head()

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109
0,0.0,-4.187822,-13.978846,-246.346827,143.771285,-33.226293,-36.133359,229.360804,42.729180,68.088766,-14.354222,379.534926,41.192372,-89.712466,91.039404,-46.293044,-248.259157,43.758404,-40.154632,-6.528808,-176.749531,61.872700,-89.548161,32.414852,-140.231821,101.054761,-66.800188,-5.896080,-18.332500,148.830528,-112.414965,54.722385,-66.320061,157.461496,-442.091978,43.226450,-34.060278,81.556107,-229.441763,234.651911,...,119.194252,29.578525,111.124966,-20.997311,40.464690,154.287036,-46.451660,-187.429135,-203.773473,225.603142,830.141659,-151.482681,7.218052,-423.338600,284.856709,-156.060589,-200.959998,92.764412,-12.277416,-162.871510,-50.166315,20.668672,44.864675,9.417400,-107.146909,296.538059,13.184251,111.088204,59.297607,-34.578166,327.181819,-61.486539,-62.615526,-67.840895,23.592680,124.809538,-23.077201,3.190311,158.660385,-83.848561
1,1.0,24.094857,-11.570042,26.793252,155.851579,89.448145,-39.356123,199.666258,95.310857,-13.326055,57.311463,710.408902,-71.270268,-51.935622,-87.875089,-57.889892,-27.943592,32.674989,22.318301,38.162048,53.924961,50.527587,-22.346857,11.142822,-98.734174,-204.123448,-105.133361,14.948635,28.605727,-20.275940,133.685980,93.595908,-14.341743,68.505643,-224.793209,-115.796889,13.006750,-207.516197,-413.299448,-26.236572,...,14.109474,-211.767700,-4.175856,61.330626,52.364868,275.311225,-5.432291,217.468878,-169.595055,-231.270777,944.409928,157.416734,-96.812772,267.488076,36.896252,-768.684009,264.122707,147.567211,-7.446930,52.083295,89.182962,-12.503631,30.669192,24.108511,95.518840,345.525334,-85.564605,93.451491,116.435131,31.777602,-73.688765,44.274265,30.681210,34.671579,66.111775,-63.125159,-43.477966,-2.787521,40.601941,-36.462574
2,2.0,129.626766,153.361044,-228.575252,-43.798189,48.125665,18.222124,-27.085524,12.712634,-14.005595,-19.269208,-114.244016,1.652489,-26.574174,-19.644541,26.151168,-73.804245,-22.481410,49.403011,115.668719,118.056089,126.248107,4.977037,-179.432432,17.780447,58.590776,102.871607,-68.585878,2.059828,-33.362586,-83.351330,127.144773,-258.129755,3.634848,-246.895556,-61.089183,-89.312486,-202.054050,274.887163,313.249016,...,11.218658,88.676689,-59.242014,20.471298,203.430250,-69.375463,95.551928,-164.583470,88.233365,-97.948005,-357.276678,-98.473885,-36.450772,-335.162340,-103.091830,-17.627212,344.520491,178.908868,62.179846,35.749772,-439.963736,-44.436014,-72.547520,166.352119,-504.351106,-208.905726,25.534120,116.166520,15.221104,-17.858483,-267.858138,14.487393,-18.822421,-72.809791,-98.768014,7.588928,34.714429,76.767352,-271.184273,63.889350
3,3.0,209.960232,3.723635,134.575274,45.502300,43.350338,475.118110,83.041198,-48.269179,12.554096,-54.057972,226.277106,-105.737869,-43.190544,93.049134,-70.907425,300.711854,13.152212,15.767445,-108.766805,-94.224412,-57.808470,60.931524,3.001858,34.700231,-121.823804,-39.880779,-66.074959,-9.245082,104.624152,-26.232481,-0.323705,63.358228,86.358240,-347.802854,122.990133,11.757166,-293.271584,-381.498203,-135.995051,...,32.403897,4.340228,-76.581513,-24.607148,99.388882,196.996405,8.510864,287.943018,51.048414,57.138122,-231.338109,-2.805622,-70.245665,2.111366,298.878915,425.049436,-274.340751,240.108552,-54.205118,-199.147496,1289.689551,25.093060,-10.294342,29.187337,406.848129,-259.466481,-15.001917,193.692069,68.706060,-31.661918,511.081948,4.706411,1.643793,-15.398200,-14.028664,65.495963,-15.017345,41.329913,-1214.900271,-25.435079
4,4.0,-179.133415,11.689463,-110.506442,138.039678,-132.239381,78.865273,288.485620,-9.575814,-67.062730,-113.020083,26.158283,-109.942460,33.816672,33.799100,93.676668,88.693241,59.256611,61.902004,-131.111820,158.971805,5.079087,47.160977,10.568292,49.772049,-186.428904,-77.385165,165.640000,-123.137159,-9.233544

In [12]:
test.head()

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109
0,0.0,70.988329,-53.726764,146.463183,-38.562153,-138.351157,-23.985445,-0.942323,41.176792,-68.501977,75.338113,151.280761,32.383098,86.354705,61.989109,-82.176753,347.949901,-20.659918,50.674314,-169.084358,-21.548420,36.701044,17.367683,71.371297,-114.220831,-120.353016,85.436424,-49.591815,-86.537309,23.828184,19.599578,13.152231,58.218131,-87.632130,156.206356,-7.995115,63.581109,-263.759296,-144.568024,-141.148836,...,28.711508,114.677544,-9.770192,151.113108,103.138697,-22.740347,67.343610,-222.158542,-39.852058,78.011262,384.510624,80.684355,-83.942262,70.938343,-168.968610,228.873753,299.943401,-113.016121,-32.710609,101.497428,-454.608652,12.442758,-49.461889,6.954776,-296.071460,-36.862581,-45.572393,-95.728081,-80.500263,-42.628110,-402.401290,-29.088708,-78.807053,-16.862015,161.805167,-50.543858,-1.989353,57.155667,349.346501,-143.966616
1,1.0,-99.136105,47.856013,-127.313282,51.535123,-14.482731,-88.489583,285.176751,-21.667084,35.069040,57.696073,-575.887613,-64.752402,25.828870,25.342384,54.761143,329.702345,-66.815945,19.848216,-44.856381,18.993308,171.798934,-129.524307,36.079756,60.550417,-50.294207,-204.474701,-88.363062,-43.746252,-29.995398,37.095432,13.427478,2.426965,17.291718,73.363014,98.012791,36.623227,92.698685,447.696768,-20.587798,...,-74.340171,56.258021,15.743743,-24.711577,35.439614,-27.336995,-88.347513,-333.056145,-11.673564,-176.962855,528.588276,-39.224135,27.171820,14.016272,127.607574,373.045611,178.043465,-159.708117,18.012749,173.999048,-403.181514,0.130585,-24.239591,-87.494270,-434.786847,-125.848540,-46.362402,-48.820081,52.422765,-2.519414,168.901890,27.334611,2.490669,-63.710021,138.582246,133.862045,56.301659,29.497996,-226.758252,-77.972184
2,2.0,-98.412507,95.380702,-103.875782,92.999655,26.714858,-31.292225,-319.328714,-87.469017,245.305830,34.611665,-635.023375,-41.587086,-77.572441,-14.560731,-18.344283,-12.043800,37.210904,-27.640646,49.793802,43.136645,152.279475,174.426519,-10.018109,49.075607,-96.851090,-353.083458,-46.354438,-38.833246,-37.499015,116.698769,-74.417050,184.041106,-99.200991,128.583684,-23.293807,5.881213,78.782842,-47.370704,27.049031,...,-77.320496,135.522983,-120.575668,-183.337851,56.328132,75.108814,-239.939668,199.203671,-244.585591,180.485070,-114.679547,50.813887,-45.639735,-111.009465,-120.271071,612.270275,90.245353,-227.492907,-22.065905,257.332478,-469.522353,-32.288949,-8.607205,140.006987,139.464368,122.808342,27.574988,-20.060605,215.693535,24.521124,-130.382216,80.051382,245.776010,33.084594,18.025982,89.830480,-173.844681,-59.906696,264.088705,-22.603129
3,3.0,21.493182,170.821211,118.465297,32.208138,40.172187,280.012980,105.452861,-35.254861,283.936144,-30.450130,-417.955713,133.622840,-26.327372,-31.934234,1.862688,-123.628311,-2.093535,37.097581,-93.457829,168.988256,1.629984,-39.572628,-13.221236,-92.315563,164.651067,-253.536852,7.861726,20.771766,64.554920,18.276642,-87.458977,-53.101199,73.524342,498.525097,63.009249,-26.447266,-238.813353,359.358358,71.078789,...,53.130559,23.247875,-102.396242,164.597209,17.635442,-48.789831,-157.662584,-373.138274,-501.978088,114.715854,721.773350,-41.615403,-73.534237,-246.111017,127.748801,282.491544,134.795973,133.154944,18.884542,274.297204,-221.445570,-44.342547,-13.889814,-109.751462,233.881825,-153.446303,78.380661,-61.583642,-10.273512,9.275184,-498.688734,-52.053109,-112.660269,42.346959,-15.023777,-162.510923,-35.469811,-51.957894,353.973430,36.174981
4,4.0,-14.428943,-88.702214,-90.949031,32.802493,-4.622240,-154.312595,131.270028,-12.524056,119.644296,24.632698,539.532158,-16.787959,20.061802,39.110959,-40.669987,-133.013243,78.306440,17.899180,-105.133154,7.334914,-95.532928,9.925823,19.846400,-84.646467,233.048472,-116.870658,55.917968,-58

In [13]:
#Нормализация данных 

scaler = StandardScaler()
X = scaler.fit_transform(X)
scaler1 = StandardScaler()
test = scaler1.fit_transform(test)
X = pd.DataFrame(X)
test = pd.DataFrame(test)


In [44]:
#Разделение данных на tran/test выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#KNN (N=5)
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn.fit(X_train, y_train)

In [ ]:
pred = knn.predict(X_test)

In [ ]:
accuracy_score(y_test, pred)

0.8678

In [ ]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=123)
dtree.fit(X_train, y_train)

In [ ]:
pred = dtree.predict(X_test)

In [ ]:
accuracy_score(y_test, pred)

0.7897

In [ ]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(n_jobs=-1)
lr.fit(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

0.674

In [ ]:
pred = lr.predict(X_test)

In [ ]:
accuracy_score(y_test, pred)

0.674

In [ ]:
#GridSearch fo LogisticRegression
from sklearn.model_selection import GridSearchCV
model = LogisticRegression(n_jobs=-1)
grid = {'penalty': ['l1', 'l2', 'elasticnet'],
        'C': [0.01, 0.1, 1, 10, 100]}
search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, scoring='accuracy', cv=5, refit=True)
search.fit(X_train, y_train)

In [ ]:
search.best_params_

{'C': 10, 'penalty': 'l2'}

In [ ]:
pred = search.predict(X_test)
accuracy_score(y_test, pred)

0.6766

In [ ]:
#GridSearch for KNN
model = KNeighborsClassifier(n_jobs=-1)
grid = {'n_neighbors': [3,5,6,7,9],
        'algorithm': ['kd_tree, ball_tree', 'brute']}
search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, scoring='accuracy', cv=5, refit=True)
search.fit(X_train, y_train)

In [ ]:
search.best_params_

{'algorithm': 'brute', 'n_neighbors': 6}

In [ ]:
pred = search.predict(X_test)
accuracy_score(y_test, pred)

0.7208

In [ ]:
#XGBoostClassifier
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimator=250, n_jobs=-1, random_state=42)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimator=250,
              n_estimators=100, n_jobs=-1, nthread=None,
              objective='multi:softprob', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [ ]:
prediction = xgb.predict(X_test)
accuracy_score(y_test, pred)

0.9218

In [ ]:
xgb = XGBClassifier(n_estimators=450, n_jobs=-1, random_state=42)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=450, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
pred = xgb.predict(X_test)
accuracy_score(y_test,pred)

0.9001

In [ ]:
#LightGBMClassifier
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(n_estimators=200, n_jobs=-1)
lgbm.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
pred = lgbm.predict(X_test)
accuracy_score(y_test, pred)

0.9378

In [18]:
#GridSearch fot LightGBMClassifier
from lightgbm import LGBMClassifier
model = LGBMClassifier(n_jobs=-1)
grid = {'n_estimators': [350, 450, 550, 650, 900]}
search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, scoring='accuracy', cv=5, refit=True)
search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'n_estimators': [350, 450, 550, 650, 900]},
             scoring='accuracy')

In [20]:
search.best_params_

{'n_estimators': 900}

In [21]:
pred = search.predict(X_test)
accuracy_score(y_test, pred)

0.9531

In [23]:
#LightGBM (n_estimators=900)
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(n_estimators=900, n_jobs=-1)
lgbm.fit(X_train,y_train)

LGBMClassifier(n_estimators=900)

In [24]:
pred = lgbm.predict(X_test)
accuracy_score(y_test, pred)

0.9574

In [31]:
#Xgboost (n_estimators=1000)
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimator=1000, n_jobs=-1, random_state=42)
xgb.fit(X_train, y_train)

XGBClassifier(n_estimator=1000, n_jobs=-1, objective='multi:softprob',
              random_state=42)

In [32]:
pred = xgb.predict(X_test)
accuracy_score(y_test, pred)

0.8098

In [25]:
#Make submission
sub = lgbm.predict(test)

In [23]:
print(sub.shape)

(10000,)


In [26]:
submission = pd.DataFrame(sub)
#submission = pd.DataFrame(submission, columns=['label'])
#print(type(submission.iloc[:,:5]))
submission.head()

,0
0,5
1,8
2,4
3,5
4,0


In [27]:
sample_sabmission = pd.read_csv('sample_sabmission.csv')
print(sample_sabmission.shape)
sample_sabmission.head()

(10000, 2)


,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [28]:
sample_sabmission['label'] = submission
sample_sabmission.head()

,id,label
0,0,5
1,1,8
2,2,4
3,3,5
4,4,0


In [29]:
sample_sabmission.to_csv('lgbm_900.csv', index=False) 